# Evaluation Entry

**Note**: This notebook generate results for evaluations. Evaluation results please go to folder 'eval_train_size'

### default nerual network parameters
~~~~
self.nn_params = {
    'hidden_size': 100,
    'depth': 3,               # network depth = 'depth' + 1
    'activation': torch.nn.Tanh,
    'device': device
}
~~~~
### default training parameters
~~~~
self.train_params = {
    'epochs': 500,
    'train_loss': [
        'mse_loss', 
        'phy_loss', 
        'energy_loss'
    ],
    'test_loss': [
        'phy_loss', 
        'energy_loss'
    ],
    'num_batch': 1,
    'optimizer': torch.optim.Adamax,
    'cyclical': {
        'base_lr': 0.001, 
        'max_lr': 0.1, 
        'step_size_up': 20, 
        'step_size_down': 20, 
        'mode': 'triangular'
    },     # set to False or {} to disable
    'L2_reg': 0.0,
    'verbose': False,
    'print_interval': 10,
    'early_stopping': False
}
~~~~
### default input/output parameters
~~~~
self.io_params = {
    'path_out': '../results/',
    'path_fig': '../figures/',
    'path_log': '../logs/',
    'env_path': [],           # path that will be registered through sys.path
    'use_timestamp': True
}
~~~~
### default dataset parameters
~~~~
self.data_params = {
    'data_path': '../datasets/',
    'phase': 'single-phase',  # use 'single-phase', 'dual-phase' or 'multi-phase'
    'n_sites': 4,
    'train_size': 20000,
    'val_size': 2000,
    'test_size': 0,           # set to 0 to use all the test data
    'normalize_input': True,
    'normalize_output': False,
    'device': device,
    'dataset': 'new'
}
~~~~
### default loss function parameters
~~~~
self.loss_params = {
    'lambda_s': 1.0,
    'lambda_e0': 0.2,
    'anneal_factor': 0.9,
    'anneal_interval': 50,    # this is a parameter also for noise and cyclical update 
    'norm_wf': False,
    # set to False or None or {} to disable the noise
    'noise': {'mode': 'uniform', 'mean': 0.0, 'var': 0.5, 'decay': 0.9}, 
    'cyclical': {'mode': 'sin', 'decay': 0.9, 'mean': 1.0, 'amp': 1.0, 'period': 20, 'cycle_momentum': False}
}
~~~~

## The Models

|           Models            |      Abbr.       |
| --------------------------- | ---------------- |
| Deep Neural Networks        | DNN              |
| Shrodinger Loss DNN         | S-DNN            |
| Energy Loss S-DNN           | SE-DNN           |
| Normalized SE-DNN           | NSE-DNN          |
| Extended Training S-DNN     | S-DNN$_{ex}$     |
| Extended Training SE-DNN    | SE-DNN$_{ex}$    |
| Normalized S-DNN$_{ex}$     | NS-DNN$_{ex}$    |
| Normalized SE-DNN$_{ex}$    | NSE-DNN$_{ex}$   |
| Label-Free S-DNN$_{ex}$     | S-DNN$_{ex}$-LB  |
| Label-Free SE-DNN$_{ex}$    | SE-DNN$_{ex}$-LB |
| Normalized S-DNN$_{ex}$-LB  | NS-DNN$_{ex}$-LB |
| Normalized SE-DNN$_{ex}$-LB | NSE-DNN$_{ex}$-LB|

In [1]:
%matplotlib inline

import seaborn as sns
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import random

import sys
sys.path.append('../scripts/')

from training import Trainer
from presets import LambdaSearch
from config_plots import global_settings
from utils import *

global_settings()

from fastprogress.fastprogress import master_bar, progress_bar

device = torch.device('cuda:0')

## Free Log Files

In [2]:
num_logs = len(
    [
        name for name in os.listdir(
            '../logs/'
        ) if os.path.isfile(name)
    ]
)

if num_logs > 2000:
    free_logs('../logs/')

## Set Up Parameters and Start Tasks

This part includes customized training parameters. Change these to whatever you need.

In [ ]:
param_presets = LambdaSearch(data_path='../datasets/')

train_sizes = [100, 200, 500, 1000, 2000, 5000, 10000, 15000, 20000]
break_loop = False
loss_plot = False
mb = master_bar(range(10))
for i in mb:
    for train_size in progress_bar(train_sizes):
        
        # the black-box neural networks
        param = param_presets.DNN()
        param.name = 'DNN'
        param.data_params['train_size'] = train_size
        param.train_params['break_loop_early'] = break_loop
        param.loss_params['lambda_s'] = 0.0
        param.loss_params['lambda_e0'] = 0.0
        param.loss_params['anneal_factor'] = 0.0
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)

        # PINN-analogue
        param = param_presets.NSE_DNNex_LB()
        param.name = 'NSE-DNNex-LF'
        param.data_params['train_size'] = train_size
        param.train_params['break_loop_early'] = break_loop
        param.loss_params['lambda_s'] = 0.566698
        param.loss_params['lambda_e0'] = 3.680050
        param.loss_params['anneal_factor'] = 0.786220
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)

        # PGNN-analogue
        param = param_presets.NSE_DNNex()
        param.name = 'NSE-DNNex'
        param.data_params['train_size'] = train_size
        param.train_params['break_loop_early'] = break_loop
        param.loss_params['lambda_s'] = 0.433467
        param.loss_params['lambda_e0'] = 2.297982
        param.loss_params['anneal_factor'] = 0.861043
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)

        # MTL-PGNN
        param = param_presets.vNSE_DNNex()
        param.name = 'vNSE-NNex'
        param.data_params['train_size'] = train_size
        param.train_params['break_loop_early'] = break_loop
        param.train_params['vanilla'] = True
        param.loss_params['lambda_s'] = 0.433467
        param.loss_params['lambda_e0'] = 2.297982
        param.loss_params['anneal_factor'] = 0.861043
        param.data_params['device'] = device
        param.nn_params['device'] = device
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)
        
        # CoPhy-PGNN
        param = param_presets.NSE_DNNex()
        param.name = 'cNSE-NNex'
        param.data_params['train_size'] = train_size
        param.train_params['break_loop_early'] = break_loop
        param.loss_params['lambda_s'] = 0.0
        param.loss_params['lambda_e0'] = 2.061856
        param.loss_params['anneal_factor'] = 0.816932
        param.data_params['device'] = device
        param.nn_params['device'] = device
        param.loss_params['cold_start'] = {
            'mode': 'sigmoid',
            'lambda_s': 0.846349,
            'threshold': 51.0,
            'smooth': 0.171778
        }    
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)

        # CoPhy-PGNN (w/o E-Loss)
        param = param_presets.NS_DNNex()
        param.name = 'cNS-NNex'
        param.data_params['train_size'] = train_size
        param.train_params['break_loop_early'] = break_loop
        param.data_params['device'] = device
        param.nn_params['device'] = device
        param.loss_params['cold_start'] = {
            'mode': 'sigmoid',
            'lambda_s': 0.846349,
            'threshold': 51.0,
            'smooth': 0.171778
        }    
        param.loss_params['lambda_s'] = 0.0
        param.loss_params['lambda_e0'] = 0.0
        param.loss_params['anneal_factor'] = 0.0
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)

        # CoPhy-PGNN (only-D_Tr)
        param = param_presets.NSE_DNN()
        param.name = 'cNSE-NN'
        param.data_params['train_size'] = train_size
        param.train_params['break_loop_early'] = break_loop
        param.data_params['device'] = device
        param.nn_params['device'] = device
        param.loss_params['lambda_s'] = 0.274606
        param.loss_params['lambda_e0'] = 3.046513
        param.loss_params['anneal_factor'] = 0.672920
        param.loss_params['cold_start'] = {
            'mode': 'sigmoid',
            'lambda_s': 0.846349,
            'threshold': 51.0,
            'smooth': 0.171778
        }
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)

        # CoPhy-PGNN (Label-free)
        param = param_presets.NSE_DNNex_LB()
        param.name = 'cNSE-NNex-LF'
        param.data_params['train_size'] = train_size
        param.train_params['break_loop_early'] = break_loop
        param.loss_params['lambda_s'] = 0.0
        param.loss_params['lambda_e0'] = 3.680050
        param.loss_params['anneal_factor'] = 0.786220
        param.data_params['device'] = device
        param.nn_params['device'] = device
        param.loss_params['cold_start'] = {
            'mode': 'sigmoid',
            'lambda_s': 0.846349,
            'threshold': 51.0,
            'smooth': 0.171778
        }    
        trainer = Trainer(master_bar=mb, plot=loss_plot)
        trainer.start(param)